# **DATA WRANGLING ITINERARIOS**
This file attemps to gather, collect, and transform the movements raw dataset from the source attached below in order to analyse the data avilable and proceed with it. Data collected dates July 2019 to December 2019 (pre-covid). The following processes will be dealt with:

1. Reading the .json files and transforming variables
2. Data exploration
3. Reshaping data
4. Filtering data

<u>Source</u>: https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=d67921bb86e64610VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

<u>Folder</u>: Datos de uso entre julio y diciembre de 2019

#### **LIBRARIES**

In [1]:
import pandas as pd 
import json
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from datetime import datetime, timedelta
from plotly.subplots import make_subplots

### **1. READ DATA and VARIABLE TRANSFORMATION**
**Dataset**: 2019XX_movements.json (X stands for the month number)    

**Description**: Dataset of the use of the electric bicycle service of the Madrid City Council.

**Dataframe size**: Observations froom july to december total 1,862,981 timestamps are colected with information on 9 variables. 

**Variables**:
- **_id**: Identificador del movimiento.
- **user_day_code**: Código del usuario. Para una misma fecha, todos los movimientos de un mismo usuario, tendrán el mismo código, con el fin de poder realizar estudios estadísticos de las tendencias diarias de los usuarios.
- **idunplug_station**: Número de la estación de la que se desengancha la bicicleta.
- **idunplug_base**: Número de la base de la que se desengancha la bicicleta.
- **idplug_station**: Número de la estación en la que se engancha la bicicleta.
- **idplug_base**: Número de la base en la que se engancha la bicicleta.
- **unplug_hourTime**: Franja horaria en la que se realiza el desenganche de la bicicleta. Se facilita la hora de inicio del movimiento, sin la información de minutos y segundos. Todos los movimientos iniciados durante la misma hora, tendrán el mismo dato de inicio.
- **travel_time**: Tiempo total en segundos, entre el desenganche y el enganche de la bicicleta.
- **track**: Detalle del trayecto realizado por la bicicleta entre la estación de partida y la de destino, en formato GeoJSON. Si existe, contendrá un elemento "FeatureCollection" que a su vez contendrá un elemento "Features" de tipo lista. Cada elemento de dicha lista consistirá en la información de un punto del trayecto y tendrá el siguiente formato:
    * **geometry**: Indica la posición de la bicicleta y contiene:
        - **type**: Tipo de la posición. Su valor siempre será "Point".
        - **coordinates**: Coordenadas de longitud y latitud de la posición.
    * **type**: tipo de elemento. Su valor siempre será "Feature".
    * **properties**: Indica las propiedades adicionales de la posición y contiene:
        - **var**: Texto con la dirección de la posición.
        - **speed**: Velocidad de la bicicleta en el momento de generarse el dato.
        - **secondsfromstart**: Segundos transcurridos desde el desenganche de la bicicleta hasta el momento de generarse el dato. Por cuestiones de anonimato, se facilitan tiempos relativos en lugar de marcas de tiempo.
- **user_type**: Número que indica el tipo de usuario que ha realizado el movimiento. Sus posibles valores son:
    - 0: No se ha podido determinar el tipo de usuario
    - 1: Usuario anual (poseedor de un pase anual)
    - 2: Usuario ocasional
    - 3: Trabajador de la empresa
- **ageRange**: Número que indica el rango de edad del usuario que ha realizado el movimiento. Sus posibles valores son:
    - 0: No se ha podido determinar el rango de edad del usuario
    - 1: El usuario tiene entre 0 y 16 años
    - 2: El usuario tiene entre 17 y 18 años
    - 3: El usuario tiene entre 19 y 26 años
    - 4: El usuario tiene entre 27 y 40 años
    - 5: El usuario tiene entre 41 y 65 años
    - 6: El usuario tiene 66 años o más
- **zip_code**: Texto que indica el código postal del usuario que ha realizado el movimiento. 

In [ ]:
itinerarios_list = ["201907", "201908", "201909", "201910", "201911", "201912"]

itinerarios = pd.DataFrame()

for i in itinerarios_list:
    data = []
    with open('../Data/Itinerarios/'+i+'_movements.json','r') as f:
        for line in f:
            data.append(json.loads(line))

    df_i = pd.json_normalize(
        data, 
        meta=['_id']
    )
    # Convert _id to format date
    df_i["unplug_hourTime"] = pd.to_datetime(df_i["unplug_hourTime"])
    # Dorp column id of the base
    itinerarios = pd.concat( [itinerarios, df_i.drop(columns=["user_day_code", "_id.$oid"])], ignore_index=True, axis=0)

itinerarios.head()

In [ ]:
itinerarios.shape

**Variables type check**:

In [ ]:
itinerarios.dtypes

Convert variables user_type, age_range, idplug_station, idunplug_station, idunplug_base and idplug_base to categorical

In [ ]:
itinerarios['user_type'] = itinerarios['user_type'].apply(str)
itinerarios['ageRange'] = itinerarios['ageRange'].apply(str)
itinerarios['idplug_station']=itinerarios['idplug_station'].astype('str')
itinerarios['idunplug_station']=itinerarios['idunplug_station'].astype('str')
itinerarios['idunplug_base']=itinerarios['idunplug_base'].astype('str')
itinerarios['idplug_base']=itinerarios['idplug_base'].astype('str')

Create variable **return_date** by adding up the unplug_houtTime and the travel_time 

In [ ]:
itinerarios["return_date"] = itinerarios["unplug_hourTime"] + pd.to_timedelta(itinerarios["travel_time"]/1440,'d')

### **2. Exploring Data and Reshaping it**

**user_type**: 93% of users are anual, 6% are employees and a 1% occasional

In [ ]:
count_type = itinerarios["user_type"].value_counts()/itinerarios.shape[0]
labels_type = count_type.index
fig = px.bar(itinerarios, x=count_type , y=labels_type, orientation='h', text=round(count_type,2) , title = "<b>Distribución del tipo de usuario</b>")
fig.show()

Most user have no determined **age**. 
- Strangely most of users who are between 0-16 years old are employees.
- Users with a defined age are mainly 27-65 years old. This is the active population. 
- Occasional users are undefined

In [ ]:
count_ages = itinerarios["ageRange"].value_counts().sort_index()
labels_ages = count_ages.index

colors = ["#0C4B76", "#1A6691", "#6BA79D", "#558C6C", "#EE860B"]
fig = go.Figure()

for i in range (4):
    fig.add_trace(go.Bar(
        y=labels_ages,
        x=itinerarios[itinerarios["user_type"] == str(i)]["ageRange"].value_counts().sort_index() ,
        name=i,
        orientation='h',
        marker=dict(
            color=colors[i],
            line=dict(color=colors[i], width=3)
        )
    ))

fig.update_layout(barmode='stack', title = "<b>Age Distribution</b> by type of user")
fig.show()

**Travel time**: has many outliers. We explore this variable further.
- There are users who don't return the bike within the same day
- There are negative travel_times. This is considered as an error
- 50% of the movements are returned in the range 7h30min and 17h30min
- Most of the bikes that are returned 

In [ ]:
itinerarios.describe()

In [ ]:
fig = px.histogram(itinerarios[(itinerarios["travel_time"] <2880) & (itinerarios["travel_time"] > 0) ], x="travel_time", title = "<b>Distribución de la duración del alquiler</b> limitando el tiempo entre 0 y 2 días")
fig.update_traces(marker_color = "#0C4B76")
fig.show()

To further explore variable **travel_time** we proceed to check what happens with bikes that are not returned before the data collection period. Are they lost?

In [ ]:
itinerarios["lost"] = np.where((pd.to_datetime(itinerarios["return_date"]).dt.tz_localize(None) > pd.to_datetime('20200101')) & (pd.to_datetime(itinerarios["unplug_hourTime"]).dt.tz_localize(None) < pd.to_datetime('20191231')) & (itinerarios["user_type"] != '3'), True, False) 
itinerarios["lost"].value_counts()

In [ ]:
itinerarios[itinerarios["lost"]==True]

As it has just been demosntrated they are returned, they are not lost. 

In [ ]:
itinerarios = itinerarios.drop(columns="lost")

Create variable **status**:
- **error**: travel time < 0
- **change_bike**: travel time is >0 & <5 mins and the plug and unplug station is the same one
- **repaired**: travel time is > 1 day and user type is an employee of BiciMAD (3)
- **long_rental**: Rental > 1 day. Travel time > 1 day and user type is 1 or 2 (normal and occasional users)
- **short_rental**: Rental < 1 day.travel time >5 & < 1 day

In [ ]:
# lo dejo por si finalmente queremos incluir la variable lost
#conditions = [
#    (itinerarios['travel_time'] < 0 ),
#    ((itinerarios['travel_time'] >= 0) & (itinerarios['travel_time'] < 5)) | ((itinerarios['travel_time'] >= 0) & (itinerarios['travel_time'] < 5) & (itinerarios['idplug_station'] == itinerarios['idunplug_station'])),
#    (itinerarios['travel_time'] >= 5) & (itinerarios['travel_time'] <= 1440),
#    (itinerarios['travel_time'] > 1440) & (itinerarios['user_type'] == '3'),
#    (itinerarios['travel_time'] > 1440) & (itinerarios['lost'] == False) & (itinerarios['user_type'] != '3') , # >1 day rental
#    (itinerarios['lost'] == True) #lost
#]
#conditions = [
#    (itinerarios['travel_time'] < 0 ),
#    ((itinerarios['travel_time'] >= 0) & (itinerarios['travel_time'] < 5)) | ((itinerarios['travel_time'] >= 0) & (itinerarios['travel_time'] < 5) & (itinerarios['idplug_station'] == itinerarios['idunplug_station'])),
#    (itinerarios['travel_time'] >= 5) & (itinerarios['travel_time'] <= 1440),
#    (itinerarios['travel_time'] > 1440) & (itinerarios['user_type'] == '3'),
#    (itinerarios['travel_time'] > 1440) & (itinerarios['lost'] == False) & (itinerarios['user_type'] != '3') , # >1 day rental
#    (itinerarios['lost'] == True) #lost
#]
#values = ['error', 'change_bike', 'successful', 'repaired', '>1_day_rental', 'lost']

conditions = [
    (itinerarios['travel_time'] < 0 ),
    ((itinerarios['travel_time'] >= 0) & (itinerarios['travel_time'] < 5)) | ((itinerarios['travel_time'] >= 0) & (itinerarios['travel_time'] < 5) & (itinerarios['idplug_station'] == itinerarios['idunplug_station'])),
    (itinerarios['travel_time'] >= 5) & (itinerarios['travel_time'] <= 1440),
    (itinerarios['travel_time'] > 1440) & (itinerarios['user_type'] == '3'),
    (itinerarios['travel_time'] > 1440) & (itinerarios['user_type'] != '3')
]
values = ['error', 'change_bike', 'short_rental', 'repaired', 'long_rental']
itinerarios["status"] =  np.select(conditions, values) 
itinerarios.head()

In [ ]:
itinerarios["status"].value_counts()

In [ ]:
colors = ["#0C4B76", "#1A6691", "#6BA79D", "#558C6C", "#EE860B"]

fig = go.Figure()

for i in range (4):
    fig.add_trace(go.Bar(
        x=itinerarios[itinerarios["user_type"] == str(i)]["status"].value_counts().sort_index() ,
        y=itinerarios[itinerarios["user_type"] == str(i)]["status"].value_counts().sort_index().index,
        orientation='h',
        marker=dict(
            color=colors[i],
            line=dict(color=colors[i], width=3)
        )
    ))

fig.update_layout(barmode='stack', title = "<b>Número de movimientos por estado</b> y tipo de usuario")
fig.show()

In [ ]:
fig = make_subplots(rows=3, cols=2)
fig.add_trace(
    go.Box(x=itinerarios[itinerarios["status"] == "short_rental"]["travel_time"], y=itinerarios[itinerarios["status"] == "short_rental"]["user_type"], name="Short Rental"), 
    row=1, col=1)
fig.add_trace(
    go.Box(x=itinerarios[itinerarios["status"] == "long_rental"]["travel_time"], y=itinerarios[itinerarios["status"] == "long_rental"]["user_type"], name="Long Rental"), 
    row=1, col=2)
fig.add_trace(
    go.Box(x=itinerarios[itinerarios["status"] == "error"]["travel_time"], y=itinerarios[itinerarios["status"] == "error"]["user_type"], name="Error"), 
    row=2, col=1)
fig.add_trace(
    go.Box(x=itinerarios[itinerarios["status"] == "change_bike"]["travel_time"], y=itinerarios[itinerarios["status"] == "change_bike"]["user_type"], name="Change Bike"), 
    row=2, col=2)
fig.add_trace(
    go.Box(x=itinerarios[itinerarios["status"] == "repaired"]["travel_time"], y=itinerarios[itinerarios["status"] == "repaired"]["user_type"], name="Repaired"), 
    row=3, col=1)
fig.update_traces(orientation='h')
fig.update_layout(title_text="<b>Distribución de tipos de usuario</b> por estado")
fig.show()

Create variable with the number of the plug and unplug station: **Origen_destino**

In [ ]:
itinerarios['Origen_destino']=itinerarios.apply(lambda x: x.idunplug_station+'-'+x.idplug_station, axis=1)

In [ ]:
itinerarios.groupby('Origen_destino')['travel_time'].agg(['count', 'mean', 'max', 'min']).sort_values('count', ascending=False)

#### Save data to CSV

In [ ]:
itinerarios.to_csv('../Data/Itinerarios/itinerarios.csv')

### **3. Reshape data**
#### Join bases and itinerarios dataframes


In [ ]:


bases= pd.read_csv("../Data/Bases/bases.csv")

Get the **plug** and **unplug** station data

In [ ]:
cols_bases=['Número', 'Distrito', 'Barrio','Número de Plazas', 'Latitud', 'Longitud']
cols_salida=[palabra+'_Salida' for palabra in cols_bases]
cols_llegada=[palabra+'_Llegada' for palabra in cols_bases]

bases['Número']=bases['Número'].astype('str')

# Left join itinerarios and bases. Vbles idunplug_station and Número
itinerarios_bases0=pd.merge(itinerarios, bases[cols_bases], how='left', left_on='idunplug_station', right_on='Número')
itinerarios_bases0.rename(columns=dict(zip(cols_bases, cols_salida)), inplace=True)

# Left join itinerarios and bases. Vbles idplug_station and Número
itinerarios_bases=pd.merge(itinerarios_bases0, bases[cols_bases], how='left', left_on='idplug_station', right_on='Número')
itinerarios_bases.rename(columns=dict(zip(cols_bases, cols_llegada)), inplace=True)

itinerarios_bases.head()

There are many NAs resulting from the join because of the Number of Station not being the same. We delete them. 

In [ ]:
itinerarios_bases.isna().sum()

In [ ]:
a=itinerarios_bases[itinerarios_bases['Número_Salida'].isna()].idunplug_station.value_counts()
for i in a.index: 
    print("-"+str(i)+"-")

In [ ]:
itinerarios_bases.drop(itinerarios_bases[itinerarios_bases['Número_Salida'].isna()].index, inplace=True)
itinerarios_bases.drop(itinerarios_bases[itinerarios_bases['Número_Llegada'].isna()].index, inplace=True)
itinerarios_bases.isna().sum()

In [ ]:
itinerarios_bases.head()

In [ ]:
itinerarios_bases.to_csv('../Data/Itinerarios/itinerarios_bases.csv')

In [ ]:
itinerarios_bases=pd.read_csv('../Data/Itinerarios/itinerarios_bases.csv')

itinerarios_bases.dtypes.index

In [ ]:
itinerarios_bases.dtypes

In [ ]:
cols_int = itinerarios_bases.select_dtypes([np.integer]).columns
cols_int
for i in cols_int: 
    itinerarios_bases[i]=pd.to_numeric(itinerarios_bases[i], downcast='integer')

cols_float = itinerarios_bases.select_dtypes([np.float]).columns
for i in cols_float:
    itinerarios_bases[i]=pd.to_numeric(itinerarios_bases[i], downcast='float')
itinerarios_bases.dtypes


In [ ]:
itinerarios_bases.drop(columns='Unnamed: 0', axis=1, inplace=True)
itinerarios_bases['zip_code']=itinerarios_bases['zip_code'].astype(str)
itinerarios_bases.to_parquet('../Data/Itinerarios/itinerarios_bases.parquet')

In [2]:
itinerarios_bases=pd.read_parquet('../Data/Itinerarios/itinerarios_bases.parquet')
situaciones = pd.read_parquet('../Data/SituacionEstaciones/situaciones.parquet')

In [3]:
situaciones.head()

,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,latitude,dock_bikes,_id,weekDay,hour
0,1,Puerta del Sol A,0,2,24,15,NaN,-3.702425,0,Puerta del Sol nº 1,40.416896,8,2019-07-01 00:29:26.018083,0,0
1,1,Puerta del Sol B,0,2,24,14,NaN,-3.702421,0,Puerta del Sol nº 1,40.417001,9,2019-07-01 00:29:26.018083,0,0
2,1,Miguel Moya,0,2,24,12,2.0,-3.705842,0,Calle Miguel Moya nº 1,40.420589,10,2019-07-01 00:29:26.018083,0,0
3,1,Plaza Conde Suchil,0,1,18,4,3.0,-3.706917,0,Plaza del Conde Suchil nº 2-4,40.430294,11,2019-07-01 00:29:26.018083,0,0
4,1,Malasaña,0,0,24,17,4.0,-3.702587,0,Calle Manuela Malasaña nº 5,40.428552,3,2019-07-01 00:29:26.018083,0,0


In [5]:
situaciones["number"].unique()

array([ nan,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  22.,  23.,  24.,
        26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.,
        37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,
        48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,
        59.,  60.,  61.,  62.,  63.,  65.,  67.,  68.,  69.,  70.,  71.,
        72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.,  81.,  82.,  83.,
        84.,  85.,  86.,  87.,  88.,  89.,  90.,  91.,  92.,  93.,  94.,
        95.,  96.,  97.,  98.,  99., 100., 101., 102., 103., 104., 105.,
       107., 108., 109., 110., 112., 113., 114., 115., 117., 118., 119.,
       120., 121., 122., 123., 124., 125., 126., 127., 128., 129., 130.,
       131., 132., 133., 134., 135., 136., 137., 138., 139., 140., 141.,
       142., 143., 144., 145., 146., 147., 148., 149., 150., 151., 152.,
       153., 154., 155., 156., 157., 158., 159., 16

In [4]:
itinerarios_bases.head()

,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,zip_code,return_date,...,Barrio_Salida,Número de Plazas_Salida,Latitud_Salida,Longitud_Salida,Número_Llegada,Distrito_Llegada,Barrio_Llegada,Número de Plazas_Llegada,Latitud_Llegada,Longitud_Llegada
0,5,1,20,154,44,5,40,2019-07-01 00:00:00+00:00,28007,2019-07-01 02:33:59.999961600+00:00,...,01-02 EMBAJADORES,21.0,40.409817,-3.703914,40,01 CENTRO,01-02 EMBAJADORES,24.0,40.413788,-3.704746
1,8,1,17,184,57,4,129,2019-07-01 00:00:00+00:00,28045,2019-07-01 03:04:00.000019200+00:00,...,01-05 UNIVERSIDAD,24.0,40.429638,-3.712818,129,07 CHAMBERÍ,07-05 RÍOS ROSAS,24.0,40.438469,-3.698282
2,15,1,16,282,53,4,43,2019-07-01 00:00:00+00:00,nan,2019-07-01 04:41:59.999971200+00:00,...,01-02 EMBAJADORES,24.0,40.408279,-3.700730,43,01 CENTRO,01-02 EMBAJADORES,24.0,40.410088,-3.702545
3,18,1,18,388,56,0,13,2019-07-01 00:00:00+00:00,nan,2019-07-01 06:27:59.999961600+00:00,...,01-06 SOL,24.0,40.418484,-3.703288,13,01 CENTRO,01-05 UNIVERSIDAD,24.0,40.427364,-3.710473
4,1,1,20,254,105,0,75,2019-07-01 00:00:00+00:00,nan,2019-07-01 04:14:00.000009600+00:00,...,04-01 RECOLETOS,24.0,40.427067,-3.687474,75,03 RETIRO,03-05 JERÓNIMOS,24.0,40.408340,-3.678415


In [ ]:
itinerarios_bases[itinerarios_bases['user_type'].isin([]) ]

In [ ]:
data = [[6, -10]]
  
# Create the pandas DataFrame
df = pd.DataFrame(data, columns=['lat', 'lon'])

fig=px.scatter_mapbox(df,lon='lon', lat='lat' ,width = 500, height = 400)

fig.update_layout(
        title='Rutas más concurridas',
        autosize=True,
        hovermode='closest',
        showlegend=True,
        width = 500,
        height = 500,
        mapbox=dict(
            bearing=0,
            center=dict(
                lat=40.425,
                lon=-3.69
            ),
            zoom=11.3,
            style= 'carto-positron' # 'open-street-map'
            
            )
        )
fig.show()


In [ ]:
import sys
print(sys.executable)

In [ ]:
itinerarios["travel_time_D"] = itinerarios["travel_time"]/1440
itinerarios["travel_time_D"].astype('timedelta64[D]')
itinerarios["travel_time_D"]= pd.to_timedelta(itinerarios["travel_time_D"],'d')
itinerarios["return_date"] = itinerarios["unplug_hourTime"] + itinerarios["travel_time_D"]
itinerarios.tail()